# 第五课 图上的其他深度学习模型

前面的课程中我们介绍了许多图神经网络模型。除了图神经网络，针对于图数据的深度学习模型还有很多，比如图上的自编码器、变分自编码器、循环神经网络和对抗生成网络等。在这一课中，我们对自编码器和变分自编码器进行代码实践。这其中包括了对模型细节和它们的应用的讲解。

## 0. 链接预测数据集

链接预测（link prediction）是常见的与图有关的任务。该任务旨在预测两个节点之间是否存在链接（link），即是否存在边。

关于链接预测的数据集，我们可以从节点分类任务的数据集直接构造。比如我们之前常用的Cora数据集，就可以无视掉它的节点标签，把Cora图里面的边当成训练/测试数据。下面我们具体来实践一下。

In [1]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import torch
import torch_geometric

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 构造一个transform，用于对数据的预处理
transform = T.Compose([
    T.NormalizeFeatures(),  # 对特征进行标准化
    T.ToDevice(device),    # 把数据放到cpu或者gpu上
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,  # 这一步很关键，是在构造链接预测的数据集
                      split_labels=True, add_negative_train_samples=False),])


dataset = Planetoid('./', name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]

下面我们来看一下具体的数据长什么样：
* 我们不需要关注y, train_mask, val_mask, test_mask；这些是节点分类里需要用到的信息。
* pos_edge_label_index是正边样本的索引，pos_edge_label是其标签（全为1）
* neg_edge_label_index是负边样本的索引，neg_edge_label是其标签（全为0）

In [2]:
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488])

In [3]:
train_data.pos_edge_label, train_data.pos_edge_label_index

(tensor([1., 1., 1.,  ..., 1., 1., 1.]),
 tensor([[2450,  279,  836,  ..., 2510,  787,  538],
         [2113,  304, 2403,  ..., 1899,   55, 1286]]))

In [4]:
val_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[263], pos_edge_label_index=[2, 263], neg_edge_label=[263], neg_edge_label_index=[2, 263])

In [5]:
val_data.neg_edge_label

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
test_data 

Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[527], neg_edge_label_index=[2, 527])

值得注意的是：
* train_data中没有自带负边样本neg_edge_label_index，因为我们会在训练过程中自己采样负样本。
* train_data和val_data里面的图是一样的（edge_index是一样的），但是他们的pos_edge_label_index（正边样本）和neg_edge_label_index（负边样本）不一样。可以看到train_data中有4488个正边样本，而val_data中只有263个正边样本（二者比例是85:5）。
* test_data中的图和train_data的图不一样了。可以看到test_data中的edge_index要多一些（多527个），527也就是test_data中的正边样本数量。

## 1. 自编码器

针对于图数据的自编码器我们称之为GAE (Graph AutoEncoder)。其包含两个组成部分，编码器（encoder）和解码器（decoder）。图上的编码器常用的就是GCN了；而解码器呢通常用一个内积来表示。具体地，给定两个节点的节点表示，解码器将计算二者的内积，其结果作为两个节点之间存在边的概率。

In [7]:
from torch_geometric.nn import GCNConv

首先构造编码器，由两层GCN组成。

In [8]:
class GCNEncoder(torch.nn.Module):
    """GCN组成的编码器"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

然后构建解码器，将给定的节点对映射到[0，1]之间，以表示边存在的概率。

In [9]:
class InnerProductDecoder(torch.nn.Module):
    """解码器，用向量内积表示重建的图结构"""
    
    def forward(self, z, edge_index, sigmoid=True):
        """
        参数说明：
        z: 节点表示
        edge_index: 边索引，也就是节点对
        """
        value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value) if sigmoid else value

In [10]:
class GAE(torch.nn.Module):
    """图自编码器。
    """
    def __init__(self, encoder, decoder=None):
        super().__init__()
        self.encoder = encoder
        self.decoder = InnerProductDecoder()

    def encode(self, *args, **kwargs): 
        """编码功能"""
        return self.encoder(*args, **kwargs)

    def decode(self, *args, **kwargs):
        """解码功能"""
        return self.decoder(*args, **kwargs)

    def recon_loss(self, z, pos_edge_index, neg_edge_index=None):
        """计算正边和负边的二值交叉熵
        
        参数说明
        ----
        z: 编码器的输出
        pos_edge_index: 正边的边索引
        neg_edge_index: 负边的边索引
        """
        EPS = 1e-15 # EPS是一个很小的值，防止取对数的时候出现0值

        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index) + EPS).mean() # 正样本的损失函数

        if neg_edge_index is None:
            neg_edge_index = torch_geometric.utils.negative_sampling(pos_edge_index, z.size(0)) # 负采样
        neg_loss = -torch.log(
            1 - self.decoder(z, neg_edge_index) + EPS).mean() # 负样本的损失函数

        return pos_loss + neg_loss

In [11]:
in_channels, out_channels = dataset.num_features, 16
model = GAE(GCNEncoder(in_channels, out_channels))

In [12]:
latent = model.encode(train_data.x, train_data.edge_index)
latent, latent.shape

(tensor([[ 7.7152e-03,  2.9030e-03,  1.5190e-03,  ..., -1.2694e-03,
          -1.2237e-03,  2.0293e-03],
         [ 1.1495e-03,  1.6908e-03,  1.7738e-03,  ...,  6.4568e-05,
           3.8351e-04, -1.9860e-03],
         [ 1.6197e-03,  2.4107e-03, -3.1520e-04,  ..., -1.1129e-03,
           2.1450e-05,  1.8590e-03],
         ...,
         [ 8.3341e-03, -5.4902e-03,  4.5780e-03,  ..., -5.8314e-03,
          -1.0872e-02,  1.1104e-03],
         [ 1.7777e-03,  3.2905e-03, -1.7124e-03,  ..., -1.5974e-03,
          -2.0170e-03,  7.0474e-04],
         [ 2.7947e-03,  3.9654e-03, -2.5049e-03,  ..., -2.2268e-03,
          -1.0494e-03, -8.5848e-04]], grad_fn=<AddBackward0>),
 torch.Size([2708, 16]))

In [13]:
model.decode(latent, train_data.edge_index)

tensor([0.5000, 0.5000, 0.5000,  ..., 0.5001, 0.5000, 0.5001],
       grad_fn=<SigmoidBackward>)

## 2. 变分自编码器

变分自编码器和自编码器基本结构相同，都是一个编码器加一个解码器。它们的主要区别是，变分自编码器编码后的隐层表示不再是连续的向量表示，而是通过一个高斯分布来表示。具体地，变分自编码器学习的是这个高斯分布的均值（下面用变量`mu`来表示）和标准差（下面用变量`std`来表示）。

In [14]:
MAX_LOGSTD = 10

class VariationalGCNEncoder(torch.nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)
    
class VGAE(GAE): 
    """变分自编码器。继承自GAE这个类，可以使用GAE里面定义的函数。
    """
    
    def __init__(self, encoder, decoder=None):
        super().__init__(encoder, decoder)

    def reparametrize(self, mu, logstd):
        if self.training:
            return mu + torch.randn_like(logstd) * torch.exp(logstd)
        else:
            return mu

    def encode(self, *args, **kwargs):
        """编码功能"""
        self.__mu__, self.__logstd__ = self.encoder(*args, **kwargs) # 编码后的mu和std表示一个分布
        self.__logstd__ = self.__logstd__.clamp(max=MAX_LOGSTD) # 这里把std最大值限制一下
        z = self.reparametrize(self.__mu__, self.__logstd__) # 进行reparametrization，这样才能够训练模型
        return z

    def kl_loss(self, mu=None, logstd=None):
        """我们给隐变量的分布加上（0，I）高斯变量的先验，即希望隐变量分布服从（0，I）的高斯分布
        这两个分布的差别用KL损失来衡量。"""
        mu = self.__mu__ if mu is None else mu
        logstd = self.__logstd__ if logstd is None else logstd.clamp(
            max=MAX_LOGSTD)
        return -0.5 * torch.mean(
            torch.sum(1 + 2 * logstd - mu**2 - logstd.exp()**2, dim=1)) # 两个高斯分布之间的KL损失

（两个高斯分布的kl loss的公式可以参考该[链接](https://stats.stackexchange.com/questions/234757/how-to-use-kullback-leibler-divergence-if-mean-and-standard-deviation-of-of-two)）

In [15]:
model = VGAE(VariationalGCNEncoder(in_channels, out_channels))
model = model.to(device)

In [16]:
latent = model.encode(train_data.x, train_data.edge_index)
latent, latent.shape

(tensor([[-0.6204, -1.5778, -0.3021,  ...,  0.3272, -0.8982,  0.7481],
         [ 0.3182,  0.6585, -1.6189,  ...,  0.7478,  0.3599,  0.2559],
         [ 0.3998, -2.1663,  1.6750,  ..., -1.6075,  0.5851,  1.0506],
         ...,
         [ 0.2672, -0.5493, -0.8153,  ...,  0.9819,  0.4806, -0.8723],
         [ 1.5876,  0.1912, -0.0117,  ..., -1.2229,  0.3848, -0.7511],
         [ 0.5664, -0.3713,  0.7488,  ..., -0.7509,  0.4346, -1.9481]],
        grad_fn=<AddBackward0>),
 torch.Size([2708, 16]))

In [17]:
model.decode(latent, train_data.edge_index)

tensor([0.1584, 0.0016, 0.0113,  ..., 0.9907, 0.2118, 0.8493],
       grad_fn=<SigmoidBackward>)

## 3. 训练自编码器和变分自编码器

接下来我们展示自编码器和变分自编码器的训练。

In [18]:
def train_gae(model):
    """训练GAE模型"""
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index)
    loss.backward()
    optimizer.step()
    return loss.item()

def train_vgae(model):
    """训练VGAE模型，损失函数由重建损失和kl损失组成"""
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index)
    loss = loss + (1 / train_data.num_nodes) * model.kl_loss() # 加上kl loss
    loss.backward()
    optimizer.step()
    return loss.item()

In [19]:
@torch.no_grad()
def test(model, data):
    """测试模型"""
    from sklearn.metrics import roc_auc_score, average_precision_score
    model.eval()
    pos_edge_index = data.pos_edge_label_index
    neg_edge_index = data.neg_edge_label_index
    
    z = model.encode(data.x, data.edge_index)
    pos_y = z.new_ones(pos_edge_index.size(1)) # 正样本标签
    neg_y = z.new_zeros(neg_edge_index.size(1)) # 负样本标签
    y = torch.cat([pos_y, neg_y], dim=0)

    pos_pred = model.decoder(z, pos_edge_index)
    neg_pred = model.decoder(z, neg_edge_index) 
    pred = torch.cat([pos_pred, neg_pred], dim=0)

    y, pred = y.detach().cpu().numpy(), pred.detach().cpu().numpy()

    return roc_auc_score(y, pred), average_precision_score(y, pred) # 计算AUC和AP

训练GAE：

In [20]:
model = GAE(GCNEncoder(in_channels, out_channels))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
 

epochs = 2000
for epoch in range(1, epochs + 1):
    loss = train_gae(model)
    if epoch % 100 == 0:
        auc, ap = test(model, test_data)
        print('Epoch: {:03d}, Loss_train: {:.4f}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, loss, auc, ap))

Epoch: 100, Loss_train: 0.9227, AUC: 0.8880, AP: 0.8941
Epoch: 200, Loss_train: 0.8615, AUC: 0.9138, AP: 0.9231
Epoch: 300, Loss_train: 0.8566, AUC: 0.9201, AP: 0.9287
Epoch: 400, Loss_train: 0.8397, AUC: 0.9229, AP: 0.9325
Epoch: 500, Loss_train: 0.8227, AUC: 0.9216, AP: 0.9327
Epoch: 600, Loss_train: 0.8273, AUC: 0.9210, AP: 0.9331
Epoch: 700, Loss_train: 0.8080, AUC: 0.9220, AP: 0.9362
Epoch: 800, Loss_train: 0.8049, AUC: 0.9247, AP: 0.9374
Epoch: 900, Loss_train: 0.8016, AUC: 0.9207, AP: 0.9347
Epoch: 1000, Loss_train: 0.7936, AUC: 0.9197, AP: 0.9360
Epoch: 1100, Loss_train: 0.7959, AUC: 0.9231, AP: 0.9384
Epoch: 1200, Loss_train: 0.7908, AUC: 0.9260, AP: 0.9429
Epoch: 1300, Loss_train: 0.7793, AUC: 0.9234, AP: 0.9427
Epoch: 1400, Loss_train: 0.7816, AUC: 0.9217, AP: 0.9419
Epoch: 1500, Loss_train: 0.7842, AUC: 0.9215, AP: 0.9408
Epoch: 1600, Loss_train: 0.7816, AUC: 0.9220, AP: 0.9409
Epoch: 1700, Loss_train: 0.7733, AUC: 0.9225, AP: 0.9405
Epoch: 1800, Loss_train: 0.7672, AUC: 0.

训练VGAE：

In [21]:
model = VGAE(VariationalGCNEncoder(in_channels, out_channels))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
 
epochs = 2000
for epoch in range(1, epochs + 1):
    loss = train_vgae(model)
    if epoch % 100 == 0:
        auc, ap = test(model, test_data)
        print('Epoch: {:03d}, Loss_train: {:.4f}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, loss, auc, ap))

Epoch: 100, Loss_train: 1.1616, AUC: 0.7363, AP: 0.7461
Epoch: 200, Loss_train: 1.0253, AUC: 0.8482, AP: 0.8456
Epoch: 300, Loss_train: 0.9404, AUC: 0.8730, AP: 0.8710
Epoch: 400, Loss_train: 0.9141, AUC: 0.8971, AP: 0.9038
Epoch: 500, Loss_train: 0.8851, AUC: 0.9088, AP: 0.9174
Epoch: 600, Loss_train: 0.8822, AUC: 0.9111, AP: 0.9208
Epoch: 700, Loss_train: 0.8721, AUC: 0.9168, AP: 0.9257
Epoch: 800, Loss_train: 0.8667, AUC: 0.9179, AP: 0.9264
Epoch: 900, Loss_train: 0.8668, AUC: 0.9163, AP: 0.9274
Epoch: 1000, Loss_train: 0.8489, AUC: 0.9186, AP: 0.9292
Epoch: 1100, Loss_train: 0.8485, AUC: 0.9212, AP: 0.9333
Epoch: 1200, Loss_train: 0.8500, AUC: 0.9204, AP: 0.9324
Epoch: 1300, Loss_train: 0.8466, AUC: 0.9253, AP: 0.9361
Epoch: 1400, Loss_train: 0.8415, AUC: 0.9237, AP: 0.9346
Epoch: 1500, Loss_train: 0.8425, AUC: 0.9252, AP: 0.9373
Epoch: 1600, Loss_train: 0.8358, AUC: 0.9278, AP: 0.9397
Epoch: 1700, Loss_train: 0.8368, AUC: 0.9247, AP: 0.9382
Epoch: 1800, Loss_train: 0.8326, AUC: 0.